******
- 针对特定问题参考材料迭代式的生成代码
- 从用户指定的一组文档开始
- 使用长上下文LLM来提取它并执行RAG来回答基于它的问题题
- 调用一个工具来生成结构化输出
- 将解决方案返回给用户之前,将执行两个单元测试,检查导入和代码执行

In [26]:
pip install langchain_community langchain-openai langchain bs4 langchain-anthropic langchain-deepseek


Note: you may need to restart the kernel to use updated packages.


#### 加载LCEL文档

In [19]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# lcel download the page
url = "https://python.langchain.com/docs/concepts/lcel/"
loader = RecursiveUrlLoader(
        url=url, 
        max_depth=20, 
        extractor=lambda x: Soup(x, "html.parser").text
    )

docs = loader.load()

d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_text = "\n\n\n --- \n\n\n".join([doc.page_content for doc in d_reversed])
print("Number of documents: ", concatenated_text)


Number of documents:  




LangChain Expression Language (LCEL) | 🦜️🔗 LangChain






Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as a retrieverHow to add memory to chatbotsHow to use example selectorsHow to add a semantic layer over graph databaseHow to invoke runnables in parallelHow to stream chat

#### 设置大模型

In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_deepseek import ChatDeepSeek
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import os

load_dotenv()

BASE_URL = os.getenv("BASE_URL")
DS_API_KEY = os.getenv("DS_API_KEY")
MODEL_API_KEY = os.getenv("MODEL_API_KEY")
MODEL_NAME = os.getenv("MODEL_NAME")

code_gen_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """
    你是一位精通LCEL【LangChain表达式语言】的编程助手。
    这里是LCEL文档的完整集合：
    --
    {context}
    --
    请根据上述提供的文档回答用户问题。确保你提供的任何代码都可以执行，
    包含所有必要的导入和已定义的变量。请按照以下结构组织你的回答：
    首先描述代码解决方案，然后列出导入语句，最后给出功能完整的代码块。
    以下是用户问题："""),
    ("placeholder", "{message}"),
])

from typing import ClassVar

class code(BaseModel):
    profile: str = Field(description="问题和解决方案的描述")
    imports: str = Field(description="代码块导入语句")
    code: str = Field(description="不包括导入语句的代码块")

llm = ChatDeepSeek(
    api_key=DS_API_KEY,
    base_url="https://api.deepseek.com",
    model="deepseek-chat",
    temperature=0.1,
    max_tokens=1000,
)

llm2 = ChatOpenAI(
    api_key=MODEL_API_KEY,
    base_url=BASE_URL,
    model=MODEL_NAME,
    temperature=0.1,
    max_tokens=1000,
)



code_gen_chain_oai = code_gen_prompt | llm.with_structured_output(code)

question = "如何使用LangChain表达式语言（LCEL）来创建一个简单的聊天机器人？"
solution = code_gen_chain_oai.invoke({
    "context": concatenated_text, 
    "message": [{"role": "user", "content": question}]
})
print(solution)
# solution

profile='使用LangChain表达式语言（LCEL）创建一个简单的聊天机器人。该机器人将接受用户输入，通过提示模板生成响应，并使用聊天模型（如ChatOpenAI）生成最终的回复。' imports='from langchain_core.prompts import ChatPromptTemplate\nfrom langchain_core.runnables import RunnablePassthrough\nfrom langchain_openai import ChatOpenAI' code='# 定义提示模板\nprompt = ChatPromptTemplate.from_template("你是一个友好的聊天机器人。请回答以下问题：{question}")\n\n# 初始化聊天模型\nmodel = ChatOpenAI()\n\n# 创建聊天机器人链\nchain = {"question": RunnablePassthrough()} | prompt | model\n\n# 调用聊天机器人\nresponse = chain.invoke("你好，你是谁？")\nprint(response.content)'


#### 定义状态

In [23]:
from typing import List
from typing_extensions import TypedDict

class GraphState(TypedDict):
    """
        图状态：
        error: 错误信息
        message: 消息列表
        generation: 代码解决方案
        iterations: 尝试次数
    """
    error: str  # 修正拼写错误，从eror改为error
    message: List
    generation: code  # 修改类型为code而不是str
    iterations: int

##### 创建节点

In [29]:
# 最大尝试次数
max_iterations = 3
# 反思
flag = "do not reflect"

def generate(state: GraphState):
        """
        生成代码解决方案

        参数：
          state (dict) 当前图状态
        返回：
           state (dict) 向状态添加新的值，generation
        """
        print("生成代码解决方案")

        # 状态
        message = state["message"]
        iterations = state["iterations"]
        error = state["error"]

        # 因错误重新路由到生成

        if error == "yes":
                message += [
                        {
                        "role": "user",
                        "content": "请重试，调用code工具来构建包含前言、导入和代码块的输出。"
                        }
                ]
        

        # 解决方案
        code_solution = code_gen_chain_oai.invoke({
            "context": concatenated_text, 
            "message": message
        })

        message += [
                {
                    "role": "assistant",
                    "content": f"{code_solution.profile} \n 导入: {code_solution.imports} \n 代码: {code_solution.code}",        
                }
        ]

        # 增加迭代次数

        iterations = iterations + 1
        return{ "generation": code_solution, "message": message, "iterations": iterations}


def code_check(state: GraphState):
        """
        检查代码解决方案

        参数：
          state (dict) 当前图状态
        返回：
           state (dict) 向状态添加新的值，error
        """
        print("检查代码解决方案")

        # 状态
        message = state["message"]
        iterations = state["iterations"]
        error = state["error"]

        # 获得解决方案组件
        code_solution = state["generation"]  # 从状态中获取code_solution
        imports = code_solution.imports
        code = code_solution.code

        # 检查导入

        try:
            exec(imports)
        except Exception as e:
            print(f"===导入导入检查:失败===")
            error_message = [{
                  "role": "user",
                  "content": f"你的解决方案未通过导入测试 \n 错误信息: {e}"
            }]
            message += error_message
            return {
                "generation": code_solution,  
                "iterations": iterations,
                "message": message,
                "error": "yes",
            }

        # 无错误
        print("===无代码测试失败===")      
        return {
            "generation": code_solution,
            "iterations": iterations,
            "message": message,
            "error": "no",
        }

def reflect(state: GraphState):
        """
        反思代码解决方案

        参数：
          state (dict) 当前图状态
        返回：
           state (dict) 向状态添加新的值，error
        """
        print("反思代码解决方案")

        # 状态
        message = state["message"]
        iterations = state["iterations"]
        error = state["error"]
        code_solution = state["generation"]  # 从状态中获取code_solution

        reflections = code_gen_chain_oai.invoke({
            "context": concatenated_text, 
            "message": message
        })

        message += [
             {
                "role": "assistant",
                "content": f"以下对错误反思: {reflections}"
             }
        ]

        return {
            "generation": code_solution,
            "iterations": iterations,
            "message": message,
        }

def decide_to_finish(state: GraphState):
        """
        决定是否完成

        参数：
          state (dict) 当前图状态
        返回：
           state (dict) 向状态添加新的值，error
        """
        print("决定是否完成")

        # 状态
        iterations = state["iterations"]
        error = state["error"]

        if error == "no" or iterations == max_iterations:
               print("===完成===")
               return "end"
        else:   
                print("===继续===")
                if flag == "reflect":
                    return "reflect"
                else:       
                    return "generate"



##### 工作流

In [32]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

workflow.add_node("generate", generate)
workflow.add_node("code_check", code_check)
workflow.add_node("reflect", reflect)

workflow.add_edge(START, "generate")
workflow.add_edge("generate", "code_check")
workflow.add_conditional_edges("code_check", decide_to_finish, {
    "end" : END,
    "reflect" : "reflect",
    "generate" : "generate",
})
workflow.add_edge("reflect", "generate")
app = workflow.compile()


In [ ]:
question = "如何再runnable中传毒原始的输入？"
solution = app.invoke({
    "message": [{"role": "user", "content": question}],
    "error": "no",
    "iterations": 0
})  

生成代码解决方案


KeyError: 'error'